In [0]:
# FUNÇÕES UTILIZADAS ==> (when + otherwise) ==> (IF/ELSE) / (like) / (isin)

# VIDEO DO YOUTUBE ==> https://www.youtube.com/watch?v=auMmIwdlFrw&list=PLOqx8yEV2-l2Oce3vUNR7L7Qjx_sy8-WO&index=4

In [0]:
# IMPORTANDO BIBLIOTECAS 

import pyspark.sql.functions
from pyspark.sql.types import StructType, StructField , StringType
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# CRIANDO SESSÃO DO SPARK 

spark = (SparkSession.builder
        .master("local(*)")
        .appName('video-4')
        .getOrCreate()
        )

# DADOS 

dados = (
    ['1','José','Anápolis','São Paulo','01-09-1900'],
    ['2','Igor','Anápolis','São Paulo','11-09-1977'],
    ['3','Leonardo','Anápolis','São Paulo','21-12-2000'],
    ['4','Humberto','Pato Branco','Rio Grande do Sul','13-11-1964'],
    ['5','Isaías','Pato Branco','Rio Grande do Sul','07-07-2002'],
    ['6','Lucas','Tauá','Ceará','05-09-1984']
    
)

# SCHEMA 

# CRIANDO COLUNAS SCHEMA DE DADOS ==> STRUCT TYPE 

colunas = (StructType([
    StructField('cod_cliente' , StringType(), True),
    StructField('nome' , StringType(), True),
    StructField('municipio' , StringType(), True),
    StructField('estado' , StringType(), True),
    StructField('data_nasc' , StringType(), True),
]))

# CRIANDO DATAFRAME FINAL 

df = spark.createDataFrame(data = dados , schema = colunas)

df.display()


cod_cliente,nome,municipio,estado,data_nasc
1,José,Anápolis,São Paulo,01-09-1900
2,Igor,Anápolis,São Paulo,11-09-1977
3,Leonardo,Anápolis,São Paulo,21-12-2000
4,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964
5,Isaías,Pato Branco,Rio Grande do Sul,07-07-2002
6,Lucas,Tauá,Ceará,05-09-1984


In [0]:
# 1) CRIE UMA COLUNA (IDADE) A PARTIR DA DATA ATUAL - A DATA_NASC 

df = df.withColumn('idade', date_format(current_timestamp() , 'yyyy').cast('integer') - substring('data_nasc' , 7,4).cast('integer'))

df.show()



+-----------+--------+-----------+-----------------+----------+-----+
|cod_cliente|    nome|  municipio|           estado| data_nasc|idade|
+-----------+--------+-----------+-----------------+----------+-----+
|          1|    José|   Anápolis|        São Paulo|01-09-1900|  122|
|          2|    Igor|   Anápolis|        São Paulo|11-09-1977|   45|
|          3|Leonardo|   Anápolis|        São Paulo|21-12-2000|   22|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|   58|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|   20|
|          6|   Lucas|       Tauá|            Ceará|05-09-1984|   38|
+-----------+--------+-----------+-----------------+----------+-----+



In [0]:
# 2) CRIE UMA COLUNA (STATUS ) ==> SE A PESSOA FOR MAIOR DE 22 ESSA COLUNA VAI FICAR (OK)

# FUNCAO IF E ELSE ==> IF = WHEN     ;  ELSE = OTHERWISE

#df = df.withColumn("status" , when(col("idade") >= 22 , "OK").otherwise("NOT"))

df = df.withColumn("status" , when(col("idade") >= 22 , lit("OK")).otherwise("NOT"))

df.display()

cod_cliente,nome,municipio,estado,data_nasc,idade,status
1,José,Anápolis,São Paulo,01-09-1900,122,OK
2,Igor,Anápolis,São Paulo,11-09-1977,45,OK
3,Leonardo,Anápolis,São Paulo,21-12-2000,22,OK
4,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964,58,OK
5,Isaías,Pato Branco,Rio Grande do Sul,07-07-2002,20,NOT
6,Lucas,Tauá,Ceará,05-09-1984,38,OK


In [0]:
# 3) CRIE UMA COLUNA (PAULISTA) E INFORME SE A PESSOA É OU NÃO PAULISTA 

df = df.withColumn('Paulista' , when(col('estado') == 'São Paulo' , "Sim").otherwise('Não'))

df.display()




cod_cliente,nome,municipio,estado,data_nasc,idade,status,Paulista
1,José,Anápolis,São Paulo,01-09-1900,122,OK,Sim
2,Igor,Anápolis,São Paulo,11-09-1977,45,OK,Sim
3,Leonardo,Anápolis,São Paulo,21-12-2000,22,OK,Sim
4,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964,58,OK,Não
5,Isaías,Pato Branco,Rio Grande do Sul,07-07-2002,20,NOT,Não
6,Lucas,Tauá,Ceará,05-09-1984,38,OK,Não


In [0]:
# 4) Crie uma coluna chamada (ADULTO) ==> para os que tem idade maior que 22 

df = df.withColumn('Adulto' , when(col('idade') > 22 , lit('sim') ).otherwise('não'))

# df = df.withColumn('Adulto' , when(col('idade') > 22 , 'sim' ).otherwise('não')) # => tambem funciona

df.show()

+-----------+--------+-----------+-----------------+----------+-----+------+--------+------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|idade|status|Paulista|Adulto|
+-----------+--------+-----------+-----------------+----------+-----+------+--------+------+
|          1|    José|   Anápolis|        São Paulo|01-09-1900|  122|    OK|     Sim|   sim|
|          2|    Igor|   Anápolis|        São Paulo|11-09-1977|   45|    OK|     Sim|   sim|
|          3|Leonardo|   Anápolis|        São Paulo|21-12-2000|   22|    OK|     Sim|   não|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|   58|    OK|     Não|   sim|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|   20|   NOT|     Não|   não|
|          6|   Lucas|       Tauá|            Ceará|05-09-1984|   38|    OK|     Não|   sim|
+-----------+--------+-----------+-----------------+----------+-----+------+--------+------+



In [0]:
# 5) CRIE UMA COLUNA (REGIÃO) E (SIGLA) E COLOQUE ESSAS INFORMAÇÕES ==> [ SÃO PAULO = SUDESTE ; SP] / [RIO GRANDE DO SUL = SUL ; RS] [CEARÁ = NORDESTE ; CE]
"""
# DESTE JEITO ABAIXO NÃO PODE ==> (SÓ PODE TER UM (WITHCOLUMN) o restante é (when)) ==> SÓ PODE TER UM (OTHERWISE) NO FIM

df = df.withColumn('Região', when(col('estado') == 'São Paulo', lit('Sudeste')).otherwise(''))\
                .withColumn('Região' , when(col('estado') == 'Rio Grande do Sul' , lit('Sul')).otherwise(''))\
                .withColumn('Região' , when(col('estado') == 'Ceará' , lit('Nordeste')).otherwise(''))\
                .withColumn('Sigla' , when(col('estado') == 'São Paulo' , lit('SP')).otherwise(''))\
                .withColumn('Sigla' , when(col('estado') == 'Rio Grande do Sul' , lit('RS')).otherwise(''))\
                .withColumn('Sigla' , when(col('estado') == 'Ceará' , lit('CE')).otherwise('')).display()


# OUTRA MANEIRA DE TRABALHAR ==> (USANDO OPERADORES LOGICOS)

df5.withColumn(“new_column”, when((col(“code”) == “a”) | (col(“code”) == “d”), “A”)
.when((col(“code”) == “b”) & (col(“amt”) == “4”), “B”)
.otherwise(“A1”)).show()

"""
df = df.withColumn('Região' , when(col('estado') == 'São Paulo' , lit('Sudeste'))\
        .when(col('estado') == 'Rio Grande do Sul' , lit('Sul'))\
        .when(col('estado') == 'Ceará' , lit('Nordeste')).otherwise('não-possui'))\
        .withColumn('Sigla' , when(col('estado') == 'São Paulo' , lit('SP'))\
                   .when(col('estado') == 'Rio Grande do Sul' , lit('RS'))\
                   .when(col('estado') == 'Ceará' , lit('CE')).otherwise('não-possui'))

df.display()

cod_cliente,nome,municipio,estado,data_nasc,idade,status,Paulista,Adulto,Região,Sigla
1,José,Anápolis,São Paulo,01-09-1900,122,OK,Sim,sim,Sudeste,SP
2,Igor,Anápolis,São Paulo,11-09-1977,45,OK,Sim,sim,Sudeste,SP
3,Leonardo,Anápolis,São Paulo,21-12-2000,22,OK,Sim,não,Sudeste,SP
4,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964,58,OK,Não,sim,Sul,RS
5,Isaías,Pato Branco,Rio Grande do Sul,07-07-2002,20,NOT,Não,não,Sul,RS
6,Lucas,Tauá,Ceará,05-09-1984,38,OK,Não,sim,Nordeste,CE


In [0]:
# 6) CRIE UMA COLUNA (CONTRATADO) SOMENTE PARA PESSOAS QUE SÃO DA (REGIÃO) ==> (SUL) E (NORDESTE)

df = df.withColumn('Contratado' , when(col('Região').isin('Sul','Nordeste') , lit('Sim')).otherwise('Não')).display()

cod_cliente,nome,municipio,estado,data_nasc,idade,status,Paulista,Adulto,Região,Sigla,Contratado
1,José,Anápolis,São Paulo,01-09-1900,122,OK,Sim,sim,Sudeste,SP,Não
2,Igor,Anápolis,São Paulo,11-09-1977,45,OK,Sim,sim,Sudeste,SP,Não
3,Leonardo,Anápolis,São Paulo,21-12-2000,22,OK,Sim,não,Sudeste,SP,Não
4,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964,58,OK,Não,sim,Sul,RS,Sim
5,Isaías,Pato Branco,Rio Grande do Sul,07-07-2002,20,NOT,Não,não,Sul,RS,Sim
6,Lucas,Tauá,Ceará,05-09-1984,38,OK,Não,sim,Nordeste,CE,Sim
